In [ ]:
pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import xlsxwriter
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# 코랩 한글깨짐 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

# 구 별 품목별 데이터 전처리

In [ ]:
data=pd.read_excel('/content/서울전역시장마트데이터(구분포함,23월포함).xlsx')
data.head()

In [ ]:
def For_product(data1, data2, data3, product) :
    cols = ['단위', '비고', '지역구', '장소','구분']
    name = ['시장', '마트', '서울']
  
    test_i = data1[data1['품목']==product]
    test_a = test_i.drop(cols, axis=1)
    test_a = test_a.groupby('조사일')['가격'].mean().reset_index()
    test_a = test_a.set_index('조사일')
    y_test = test_a['가격'].resample('MS').mean()
    y_test = y_test['2017' : ]
    y = pd.DataFrame(y_test)

    test_0_i = data2[data2['품목']==product]
    test_0_a = test_0_i.drop(cols, axis=1)
    test_0_a = test_0_a.groupby('조사일')['가격'].mean().reset_index()
    test_0_a = test_0_a.set_index('조사일')
    x_test = test_0_a['가격'].resample('MS').mean()
    x_test=x_test['2017' :]
    x = pd.DataFrame(x_test)
    
    seoul = data3[data3['품목']==product]
    seoul = seoul.drop(cols, axis=1)
    seoul = seoul.groupby('조사일')['가격'].mean().reset_index()
    seoul = seoul.set_index('조사일')
    seoul = seoul['가격'].resample('MS').mean()
    seoul = seoul['2017' : ]
    z = pd.DataFrame(seoul)

    do_merge = y.merge(x,on='조사일').merge(z,on='조사일')
    do_merge.columns = name
    do_merge = do_merge.reset_index()
#     do_merge = do_merge.set_index('조사일')
#     print(do_merge)
# do_merge = pd.merge(do_merge, z, left_on=do_merge.index, right_on=z.index)
    
# #     do_merge.columns = name
    
    return (do_merge, product)

In [ ]:
def ForAllProduct(data, district):
    test_market = data[(data['지역구']==district)&(data['구분']=='시장')]
    test_outlet = data[(data['지역구']==district)&(data['구분']=='마트')]
    seoul = data[(data['지역구']==district)]
    test_list = test_market['품목'].unique()
    print(test_list)

    if len(test_list) == 5:
        (test01, product1) = For_product(test_market, test_outlet, seoul, test_list[0])
        (test02, product2) = For_product(test_market, test_outlet, seoul, test_list[1])
        (test03, product3) = For_product(test_market, test_outlet, seoul, test_list[2])
        (test04, product4) = For_product(test_market, test_outlet, seoul, test_list[3])
        (test05, product5) = For_product(test_market, test_outlet, seoul, test_list[4])
        (test06, product6) = ("There is Only 5 products exist", "Nothing")
    elif len(test_list) == 6:
        (test01, product1) = For_product(test_market, test_outlet, seoul, test_list[0])
        (test02, product2) = For_product(test_market, test_outlet, seoul, test_list[1])
        (test03, product3) = For_product(test_market, test_outlet, seoul, test_list[2])
        (test04, product4) = For_product(test_market, test_outlet, seoul, test_list[3])
        (test05, product5) = For_product(test_market, test_outlet, seoul, test_list[4])
        (test06, product6) = For_product(test_market, test_outlet, seoul, test_list[5])
        
    return test01, test02, test03, test04, test05, test06

In [ ]:
금천구_배추, 금천구_무, 금천구_양파, 금천구_상추, 금천구_오이, test06= ForAllProduct(data, '금천구')

In [ ]:
금천구_배추

In [ ]:
서울배추=금천구_배추[['조사일','서울']]
서울배추.rename(columns={'조사일': '조사일', '서울': '서울배추'}, inplace=True)
서울배추.set_index('조사일',inplace=True)

서울무=금천구_무[['조사일','서울']]
서울무.rename(columns={'조사일': '조사일', '서울': '서울무'}, inplace=True)
서울무.set_index('조사일',inplace=True)

서울양파=금천구_양파[['조사일','서울']]
서울양파.rename(columns={'조사일': '조사일', '서울': '서울양파'}, inplace=True)
서울양파.set_index('조사일',inplace=True)

서울상추=금천구_상추[['조사일','서울']]
서울상추.rename(columns={'조사일': '조사일', '서울': '서울상추'}, inplace=True)
서울상추.set_index('조사일',inplace=True)

서울오이=금천구_오이[['조사일','서울']]
서울오이.rename(columns={'조사일': '조사일', '서울': '서울오이'}, inplace=True)
서울오이.set_index('조사일',inplace=True)

In [ ]:
서울평균채소가격=pd.concat([서울배추,서울무,서울양파,서울상추,서울오이],axis=1)
서울평균채소가격.to_excel('서울평균채소가격.xlsx',index=False)
서울평균채소가격.reset_index('조사일',inplace=True)
서울평균채소가격.head()

In [ ]:
서울품목별가격변동=pd.read_excel('/content/서울품별목별가격변동.xlsx')
#서울품목별가격변동.to_excel('서울품목별가격변동.xlsx',index=False)
#서울품목별가격변동.set_index('조사일',inplace=True)
서울품목별가격변동.head()

# 양파 변동률 그래프

In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# loading dataset 

서울품목별가격변동=pd.read_excel("/content/서울품별목별가격변동.xlsx")
서울품목별가격변동.head()
# draw lineplot 
plt.figsize=(50,50)
sns.lineplot(x="조사일", y="양파monthChange_rate", data=서울품목별가격변동)
plt.xlabel('날짜')
plt.ylabel('양파가격변동률')
plt.title('서울지역 양파 가격 변동률 그래프')

# 지도 시각화

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
###################### 지도 시각화
import json
with open('/content/서울경계데이터.json','r') as f:
    json_data=json.load(f)
print(json.dumps(json_data))

In [ ]:
import folium
m=folium.Map(location=[37.5838699,127.0565831],zoom_start=10)

import json
with open('/content/서울경계데이터.json','r') as f:
    geo=json.loads(f.read())
    f.close()

folium.GeoJson(
    geo,
    name='seoul'
).add_to(m)

m.save('map.html')

In [ ]:
#html 코드 
folium.Map().get_root().render()

In [ ]:
지도시각화_df=pd.read_excel('/content/지도 시각화_ 색깔별로 나눌거다 ㅡㅡ.xlsx')
무지도시각화_df=지도시각화_df[지도시각화_df['품목']=='무']
무지도시각화_df=무지도시각화_df[['구','가격']]
무지도시각화_df.head()

In [ ]:
# 서울시 중심부의 위도, 경도 입니다.
center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center, zoom_start=10)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=geo,
    data=무지도시각화_df,
    columns=("구","가격"),
    key_on='feature.properties.name',
    fill_color='BuPu'
).add_to(m)

# 맵 m을 출력합니다.
m

In [ ]:
# 무 관련 한거임
지도시각화_plz=pd.read_excel('/content/최종최종 최종 지도시각화 위도경도 있는 데이터다 ㅡㅡ.xlsx')
지도시각화_plz=지도시각화_plz[지도시각화_plz['품목']=='무']
지도시각화_plz=지도시각화_plz[['장소','평균 : 4년 평균 가격','위도','경도']]
지도시각화_plz.columns=['장소','평균가격','위도','경도']
지도시각화_plz.head()

In [ ]:
import folium as g

In [ ]:
for name,price,lat,lon in zip(지도시각화_plz.장소,지도시각화_plz.평균가격,지도시각화_plz.위도, 지도시각화_plz.경도):
    g.Marker([lat, lon], 
             popup= name+str(price),
             icon=g.Icon(color='red')).add_to(m)
m0

# 가락시장데이터(경락가예측 전처리)




In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
배추_상_가격데이터=pd.read_excel('/content/배추(상, 10kg) 가락시장 가격 데이터.xlsx')
배추_특_가격데이터=pd.read_excel('/content/배추(특, 10kg) 가락시장 가격 데이터.xlsx')
배추_반입량=pd.read_excel('/content/복사본 배추 반입량(2017~20201031).xlsx')
배추_강릉_기상=pd.read_excel('/content/복사본 배추_강릉_기상_20170101_20201031.xlsx')

In [ ]:
배추_상_가격데이터['일자']= 배추_상_가격데이터['일자'].astype('str')
배추_특_가격데이터['일자']= 배추_특_가격데이터['일자'].astype('str')
배추_상_가격데이터['일자'] = pd.to_datetime(배추_상_가격데이터['일자'])
배추_특_가격데이터['일자']= pd.to_datetime(배추_특_가격데이터['일자'])
배추_반입량['거래일자']= 배추_반입량['거래일자'].astype('str')
배추_반입량['거래일자'] = pd.to_datetime(배추_반입량['거래일자'])
배추_반입량.rename(columns = {'거래일자' : '일자'}, inplace = True)
배추_강릉_기상.rename(columns = {'일시' : '일자'}, inplace = True)
배추_강릉_기상.head()
배추_상_dt=pd.merge(pd.merge(배추_상_가격데이터,배추_반입량, on='일자'),배추_강릉_기상, on='일자')
배추_상_dt.head()
배추_특_dt=pd.merge(pd.merge(배추_특_가격데이터,배추_반입량,on='일자'),배추_강릉_기상,on='일자')
배추_특_dt.head()
배추_상_dt.to_excel('배추_상_dt.xlsx',index=False)
배추_특_dt.to_excel('배추_특_dt.xlsx',index=False)

In [ ]:
##########양파########

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
양파_상_가격데이터=pd.read_excel('/content/양파 (상, 20kg) 가락시장 가격 데이터.xlsx')
양파_특_가격데이터=pd.read_excel('/content/양파 (특, 20kg) 가락시장 가격 데이터.xlsx')
양파_반입량=pd.read_excel('/content/복사본 양파 반입량(2017~20201031).xlsx')
양파_해남_기상=pd.read_excel('/content/복사본 양파_해남_기상_20170101_20201031.xlsx')

In [ ]:
# 가격데이터 일자 컬럼 -> 날짜데이터로 변환
양파_상_가격데이터['일자'] = 양파_상_가격데이터.astype(str)
양파_상_가격데이터['일자'] = pd.to_datetime(양파_상_가격데이터['일자'],errors='coerce')
양파_특_가격데이터['일자'] = 양파_특_가격데이터.일자.astype(str)
양파_특_가격데이터['일자'] = pd.to_datetime(양파_특_가격데이터['일자'],errors='coerce')
# 기상 데이터도 일자 변환해주기
양파_해남_기상['일시'] = 양파_해남_기상.일시.astype(str)
양파_해남_기상['일시'] = pd.to_datetime(양파_해남_기상['일시'],errors='coerce')
# 반입량 데이터도 일자 변환해주기
양파_반입량['거래일자'] = 양파_반입량.거래일자.astype(str)
양파_반입량['거래일자'] = pd.to_datetime(양파_반입량['거래일자'],errors='coerce')

In [ ]:
양파_반입량.rename(columns={'거래일자' : '일자'}, inplace=True)
양파_해남_기상.rename(columns={'일시' : '일자'}, inplace=True)

In [ ]:
양파_상_dt = pd.merge(pd.merge(양파_상_가격데이터, 양파_해남_기상, on='일자', ),양파_반입량, on='일자')
양파_상_dt.to_excel('양파_상_dt.xlsx',index=False)

In [ ]:
양파_특_dt = pd.merge(pd.merge(양파_특_가격데이터, 양파_해남_기상, on='일자', ),양파_반입량, on='일자')
양파_특_dt.to_excel('양파_특_dt.xlsx',index=False)

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
#from google.colab import files
#uploaded=files.upload()

In [ ]:
양파_특=pd.read_excel('/content/양파_특(20kg)_dt.xlsx')
양파_상=pd.read_excel('/content/양파_상(20kg)_dt.xlsx')
배추_특=pd.read_excel('/content/배추_특(10kg)_dt.xlsx')
배추_상=pd.read_excel('/content/배추_상(10kg)_dt.xlsx')

In [ ]:
del 양파_특['일자']

In [ ]:
양파_특=양파_특[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량']]
양파_특corr = 양파_특.corr(method = 'pearson') 
#양파_특corr

In [ ]:
df_heatmap = sns.heatmap(양파_특corr, cbar = True, annot = True, annot_kws={'size' : 10}, fmt = '.2f', square = True, cmap = 'Blues')
df_heatmap

In [ ]:
import scipy.stats as ss 
양파_특_ss=ss.zscore(양파_특)
print(양파_특_ss)

In [ ]:
pd.DataFrame(양파_특_ss)

In [ ]:
feature_columns=list(양파_특.columns.difference(['평균가격']))
X=양파_특[feature_columns]
y=양파_특.평균가격
train_x,test_x,train_y,test_y=train_test_split(X,y,train_size=0.7,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
#Train the MLR/회귀모델적합
full_model=sm.OLS(train_y,train_x)
fitted_full_model=full_model.fit()

In [ ]:
fitted_full_model.summary()

In [ ]:
# VIF를 통한 다중공선성 확인 
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_특.values,i) for i in range(양파_특.shape[1])]
vif['features']=양파_특.columns
vif

In [ ]:
# 변수선택법 
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)]) # modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {'model' : regr, 'AIC' : AIC}

print(processSubset(X=train_x, y = train_y, feature_set = feature_columns[0:5]))

In [ ]:
# 전진선택법(step=1)

def forward(X, y, predictors):
    # 데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predcitors+[p]+['const']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:',best_model[0])
    return best_model

In [ ]:
# 전진선택법 모델
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=['AID', 'model'])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0 ~ 9  -> 1 ~ 10
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [ ]:
#Forward_best_model = forward_model(X=train_x, y=train_y)

 # Prophet 시계열 예측

In [ ]:
서울_전역=pd.read_excel('/content/서울_4년_평균.xlsx')
서울_전역.head()

In [ ]:
강남_양파_dt=서울_전역[(서울_전역['지역구']=='강남구')&(서울_전역['품목']=='양파')]
강남_양파_dt=강남_양파_dt[['조사일','시장']]
강남_양파_dt.set_index('조사일')
강남_양파_dt.info()

In [ ]:
from matplotlib import pyplot

In [ ]:
강남_양파_dt.plot(x='조사일')
plt.show()
# 컬럼명 변경
강남_양파_dt.columns = ['ds', 'y']

# 데이터 타입 변경
강남_양파_dt['ds']= pd.to_datetime(강남_양파_dt['ds'])

# 모델 생성
from fbprophet import Prophet
model = Prophet()

# 모델 학습
model.fit(강남_양파_dt)

# train set 마지막 1년 날짜 생성
last_1year = list()
for i in range(1,12):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

# 에측
forecast = model.predict(last_1year)

forecast.columns

print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
model.plot(forecast)
pyplot.show()

# train set 이후 10개월 날짜 생성
last_1year = list()
for i in range(1, 10):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

forecast = model.predict(last_1year)
model.plot(forecast)
pyplot.show()

# 마지막 12개월 제외시키기
train = 강남_양파_dt.drop(강남_양파_dt.index[-9:])
y_true = 강남_양파_dt['y'][-9:].values

# 모델 생성 후 학습
model = Prophet()
model.fit(train)

# train set 마지막 1년 날짜 생성
last_1year = list()
for i in range(1, 10):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

# 예측하고 비교하기
# 여기에서는 MAE를 살펴본다.
forecast = model.predict(last_1year)
y_pred = forecast['yhat'].values
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

pyplot.plot(y_true, label='Actual')
pyplot.plot(y_pred, label='Predicted')
pyplot.legend()
pyplot.show()

# ARIMA 시계열 예측

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings
import itertools # 반복 가능한 데이터 스트림을 처리하는 데 유용한 많은 함수와 제네레이터가 포함
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm #통계분석 기능을 제공하는 파이썬 패키지
import matplotlib

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
서울_전역=pd.read_excel('/content/서울_4년_평균.xlsx')
서울_전역.head()
강남_양파_dt=서울_전역[(서울_전역['지역구']=='강남구')&(서울_전역['품목']=='양파')]
강남_양파_dt.head()
강남_양파_dt=강남_양파_dt[['조사일','시장']]
강남_양파_dt.head()
강남_양파_dt=강남_양파_dt.set_index('조사일')
#강남_양파_dt=강남_양파_dt.iloc[0:46,0]
#강남_양파_dt.tail()

In [ ]:
from matplotlib import pyplot

In [ ]:
강남_양파_dt.plot()
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults

model=ARIMA(강남_양파_dt,order=(1,1,1))
model_fit=model.fit(trend='c',full_output=True,disp=1)
print(model_fit.summary())
model_fit.plot_predict()
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))


for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(강남_양파_dt,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults

model=ARIMA(강남_양파_dt,order=(1,1,1), seasonal=(1,1,0,12))
model_fit=model.fit(trend='nc',full_output=True,disp=1)
print(model_fit.summary())
model_fit.plot_predict()
fore = model_fit.forecast(steps=1)
print(fore)

#경락가 다중회귀분석 & 변수선택법(1)

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
#from google.colab import files
#uploaded=files.upload()

In [ ]:
양파_특=pd.read_excel('/content/양파_특(20kg)_dt.xlsx')
양파_상=pd.read_excel('/content/양파_상(20kg)_dt.xlsx')
배추_특=pd.read_excel('/content/배추_특(10kg)_dt.xlsx')
배추_상=pd.read_excel('/content/배추_상(10kg)_dt.xlsx')

In [ ]:
del 양파_특['일자']

In [ ]:
양파_특=양파_특[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량']]
양파_특corr = 양파_특.corr(method = 'pearson') 
#양파_특corr

In [ ]:
df_heatmap = sns.heatmap(양파_특corr, cbar = True, annot = True, annot_kws={'size' : 10}, fmt = '.2f', square = True, cmap = 'Blues')
df_heatmap

In [ ]:
import scipy.stats as ss 
양파_특_ss=ss.zscore(양파_특)
print(양파_특_ss)

In [ ]:
pd.DataFrame(양파_특_ss)

In [ ]:
feature_columns=list(양파_특.columns.difference(['평균가격']))
X=양파_특[feature_columns]
y=양파_특.평균가격
train_x,test_x,train_y,test_y=train_test_split(X,y,train_size=0.7,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
#Train the MLR/회귀모델적합
full_model=sm.OLS(train_y,train_x)
fitted_full_model=full_model.fit()

In [ ]:
fitted_full_model.summary()

In [ ]:
# VIF를 통한 다중공선성 확인 
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_특.values,i) for i in range(양파_특.shape[1])]
vif['features']=양파_특.columns
vif

In [ ]:
# 변수선택법 
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)]) # modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {'model' : regr, 'AIC' : AIC}

print(processSubset(X=train_x, y = train_y, feature_set = feature_columns[0:5]))

In [ ]:
# 전진선택법(step=1)

def forward(X, y, predictors):
    # 데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predcitors+[p]+['const']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:',best_model[0])
    return best_model

In [ ]:
# 전진선택법 모델
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=['AID', 'model'])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0 ~ 9  -> 1 ~ 10
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [ ]:
#Forward_best_model = forward_model(X=train_x, y=train_y)

# 경락가 다중회귀분석 & 변수선택법(2)

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
양파_dt.info()

In [ ]:
# 설명변수(X),타겟변수(Y) 분리 및 학습 데이터와 평가 데이터 분할
feature_columns=list(양파_dt.columns.difference(['평균가격']))
X=양파_dt[feature_columns]
y=양파_dt.평균가격
train_x,test_x,train_y,test_y=train_test_split(X,y,train_size=0.7,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
#Train the MLR/회귀모델적합
full_model=sm.OLS(train_y,train_x)
fitted_full_model=full_model.fit()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
# 변수선택법 
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)]) # modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {'model' : regr, 'AIC' : AIC}

print(processSubset(X=train_x, y = train_y, feature_set = feature_columns[0:5]))

In [ ]:
# 전진선택법(step=1)

def forward(X, y, predictors):
    # 데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['평균가격']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predcitors+[p]+['평균가격']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:',best_model[0])
    return best_model

In [ ]:
# 전진선택법 모델
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=['AID', 'model'])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0 ~ 9  -> 1 ~ 10
    for i in range(1, len(X.columns.difference(['평균가격'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != '평균가격']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [ ]:
#Forward_best_model = forward_model(X=train_x, y=train_y)

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격','평균가격']]
양파_dt.head()

In [ ]:
from sklearn import linear_model

X_data=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격']]
Y_data=양파_dt[['평균가격']]

In [ ]:
#다중선형회귀모델 설계
linear_regression_model=linear_model.LinearRegression()
linear_regression_model.fit(X_data,Y_data)
linear_regression_model_prediction=linear_regression_model.predict(X=pd.DataFrame(X_data))

In [ ]:
#Train the MLR / 회귀모델적합
fullModel = sm.OLS(train_y,train_x)
fittedFullModel = fullModel.fit()
 
#R-Squre 가 높고 , 대부분의 변수들이 유의함.
print(fittedFullModel.summary())
 
#VIF를 통한 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
 
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(양파_dt.values,i)
                     for i in range(양파_dt.shape[1])]
vif["features"]=양파_dt.columns
print(vif)


In [ ]:
#학습데이터의 잔차 확인
 
res = fittedFullModel.resid
import matplotlib.pyplot as plt
#Q-Q plot # 정규분포확인
fig = sm.qqplot(res, fit=True, line='45')
plt.show()
# residual pattern 확인
 
predY = fittedFullModel.predict(train_x)
 
fig = plt.scatter(predY,res,s=4)
plt.xlim(4000,30000)
plt.xlim(4000,30000)
plt.xlabel('Fitted values')
plt.ylabel('Residual')
plt.show()
 
#검증 데이터에 대한 예측
 
predY2 = fittedFullModel.predict(test_x)
 
plt.plot(np.array(test_y-predY2),label="predFull")
plt.legend()
plt.show()
 
#MSE 값 구하기
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true=test_y,y_pred=predY2)
print(MSE)

# 데이터 시각화

####요일별 평균 가격비교

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
무_상=pd.read_excel('/content/무_상.xlsx')
무_특=pd.read_excel('/content/무_특.xlsx')
#무_상.head()
무_상.info()  # 14개의 컬럼과 2090개의 index를 가지고 있음
무_특.info() # 14개의 컬럼과 2090개의 index를 가지고 있음

In [ ]:
무_상_요일별평균가격=무_상[['평균가격','요일(1)']]
무_상_요일별평균가격.head()
무_상_요일별평균가격.groupby(무_상_요일별평균가격['요일(1)']).mean('평균가격')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Basic Bar Chart
plt.plot()
sns.barplot(x='요일(1)',y='평균가격',data=무_상_요일별평균가격)
plt.title('무_상_요일 별 평균가격')

In [ ]:
무_특_요일별평균가격=무_특[['평균가격','요일(1)']]
무_특_요일별평균가격.head()
무_특_요일별평균가격.groupby(무_특_요일별평균가격['요일(1)']).mean('평균가격'

In [ ]:
plt.plot()
sns.barplot(x='요일(1)',y='평균가격',data=무_특_요일별평균가격)
plt.title('무_특_요일 별 평균가격')

#### 가격 분포 히스토그램

In [ ]:
무_상.describe()

In [ ]:
plt.hist(무_상['평균가격'], range=(2000,40000))
plt.title('무상 평균가격대 분포 히스토그램')

In [ ]:
무_특.describe()

In [ ]:
plt.hist(무_특['평균가격'], range=(4000,45000))
plt.title('무_특 평균가격대 분포 히스토그램')

#### 날씨 상관관계

In [ ]:
무_상_기상상관관계=무_상[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','일교차']]

In [ ]:
#heatmap으로 상관관계를 표시
import seaborn as sb
plt.rcParams["figure.figsize"] = (5,5)
sb.heatmap(무_상_기상상관관계.corr(),
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = -1, vmax=1 , #컬러차트 영역 -1 ~ +1
          )

In [ ]:
무_특_기상상관관계=무_특[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','일교차']]

In [ ]:
sns.pairplot(무_특_기상상관관계)

### 반입량 평균가격 산점도

In [ ]:
# 무_상 평균가격 반입량 산점도
# Basic Scatter Plot

plt.plot('평균가격',  # x 
         '반입량',  # y
         data=무_상, 
         linestyle='none', 
         marker='o', 
         markersize=2,
         color='blue', 
         alpha=0.5)
plt.title('무_상 평균가격_반입량 산점도')
plt.show()

In [ ]:
# 무_특 평균가격 반입량 산점도
# Basic Scatter Plot

plt.plot('평균가격',  # x 
         '반입량',  # y
         data=무_특, 
         linestyle='none', 
         marker='o', 
         markersize=2,
         color='blue', 
         alpha=0.5)
plt.title('무_특 평균가격_반입량 산점도')
plt.show()

### 월별평균가격과 반입량 그래프

In [ ]:
무_상.set_index('일자',inplace=True)
무상_평균가격= 무_상[['평균가격','반입량']].resample('MS').mean()
무상_평균가격
무_특.set_index('일자',inplace=True)
무_특_평균가격= 무_특[['평균가격','반입량']].resample('MS').mean()
무_특_평균가격

In [ ]:
from matplotlib import pyplot as plt
a=np.linspace(0,5,200)

ax=무_특_평균가격.plot(kind='line', x='일자', y='평균가격', color='DarkBlue')

ax2=무_특_평균가격.plot(kind='line', x='일자', y='반입량', secondary_y=True, color='Red', ax=ax)

ax.set_ylabel('반입량')
ax2.set_ylabel('평균가격')
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(무_특_평균가격['평균가격'], color='green', linestyle='-',label='평균가격')
plt.plot(무_특_평균가격['반입량'], color='blue',  linestyle='-',label='반입량')
plt.legend()
plt.title('무_특 2013~2020 월별 평균가격과 반입량 추세 그래프 ')
plt.show()

# 경락가 예측 XGBRegressor(1)

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
양파_반입량=pd.read_excel('/content/양파 반입량 통합2013-2020.xlsx')
양파_경락가=pd.read_excel('/content/양파(특) 경락가 통합2013-2020.xlsx')
양파_해남_기상=pd.read_excel('/content/양파_해남20130101_20201031.xlsx')
주유소_판매가격=pd.read_excel('/content/주유소_평균판매가격.xlsx')
양파_반입량.head()

In [ ]:
양파_경락가.info()

In [ ]:
양파_경락_반입=pd.merge(양파_반입량,양파_경락가,on='일자')
양파_경락_반입.head()
양파_경락_반입['일자'] = 양파_경락_반입['일자'].astype(str)
양파_경락_반입['일자'] = pd.to_datetime(양파_경락_반입['일자'])
양파_경락_반입.info()
#양파_해남_기상.head()

In [ ]:
양파_해남_기상.rename(columns = {'일시': '일자'}, inplace = True)
양파_dt=pd.merge(양파_해남_기상,양파_경락_반입,on='일자')
양파_dt=pd.merge(양파_dt,주유소_판매가격,on='일자')
양파_dt[['일교차']]=양파_dt['최고기온(°C)']-양파_dt['최저기온(°C)']
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량','유가 전국평균가격','일교차','평균가격']]

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:
X, y = 양파_dt.iloc[:,:-1],양파_dt.iloc[:,-1]

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results.head()
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
import matplotlib.pyplot as plt

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [40, 10]
plt.show()
print((cv_results["test-rmse-mean"]).tail(1))
plt.rc('font', family='NanumBarunGothic') 

# 변수중요도
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

# 경락가 예측 XGBRegressor(2)

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
서울_양파_가격=pd.read_excel('/content/서울전역시장마트데이터(구분포함,23월포함).xlsx')
양파_특_가락시장_data=pd.read_excel('/content/양파_특.xlsx')
서울날씨=pd.read_excel('/content/seoul_weather.xlsx')
공휴일=pd.read_excel('/content/holiday.xlsx')

In [ ]:
서울_양파_가격.head()
서울날씨.rename(columns = {'일시':'일자'}, inplace = True)
서울날씨.head()
공휴일.head()

In [ ]:
서울_양파_가격=서울_양파_가격[(서울_양파_가격['품목']=='양파')&(서울_양파_가격['구분']=='마트')]
서울_양파_가격=서울_양파_가격.groupby('조사일').mean()
서울_양파_가격.reset_index('조사일',inplace = True)
서울_양파_가격.rename(columns = {'조사일':'일자'}, inplace = True)
양파_특_가락시장_data.rename(columns = {'평균가격':'경락가평균가격'}, inplace = True)
양파_통합 =pd.merge(서울_양파_가격, 양파_특_가락시장_data, on='일자')
양파_통합=pd.merge(양파_통합,공휴일,on='일자')
양파_통합=pd.merge(양파_통합,서울날씨,on='일자')
양파_통합.to_excel('양파_마트통합.xlsx')
양파_통합=양파_통합[['가격','반입량','경락가평균가격','유가 전국평균가격','유무','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
양파_통합.data=양파_통합[['반입량','경락가평균가격','유가 전국평균가격','최저기온(°C)','최고기온(°C)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 5cm 지중온도(°C)']]

In [ ]:
#양파_통합.data=양파_통합[['반입량','경락가평균가격','유가 전국평균가격','평균 풍속(m/s)']]

In [ ]:
양파_통합.target=양파_통합[['가격']]
x,y=양파_통합.data,양파_통합.target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
xgbr.fit(xtrain, ytrain)
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))


In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

# 경락가 예측 XGBRegressor(3)

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
서울_무_가격=pd.read_excel('/content/서울전역시장마트데이터(구분포함,23월포함).xlsx')
무_특_가락시장_data=pd.read_excel('/content/무_특.xlsx')
서울날씨=pd.read_excel('/content/seoul_weather.xlsx')
공휴일=pd.read_excel('/content/holiday.xlsx')

In [ ]:
무_도매=pd.read_excel('/content/서울 평균 도매가격(무).xlsx')
무_도매['서울평균도매가격']=무_도매['서울평균도매가격']/20
무_도매=무_도매[['일자','서울평균도매가격']]

In [ ]:
서울날씨.rename(columns = {'일시':'일자'}, inplace = True)
서울날씨.head()
#공휴일.head()

In [ ]:
서울_무_가격=서울_무_가격[(서울_무_가격['품목']=='무')&(서울_무_가격['구분']=='시장')]
서울_무_가격=서울_무_가격.groupby('조사일').mean()
서울_무_가격.reset_index('조사일',inplace = True)
서울_무_가격.rename(columns = {'조사일':'일자'}, inplace = True)
서울_무_가격.info()

In [ ]:
무_특_가락시장_data.rename(columns = {'평균가격':'경락가평균가격'}, inplace = True)
무_특_가락시장_data=무_특_가락시장_data[['일자','경락가평균가격','반입량','유가 전국평균가격','일교차']]
무_특_가락시장_data['경락가평균가격']=무_특_가락시장_data['경락가평균가격']/20

In [ ]:
무_통합 =pd.merge(서울_무_가격, 무_특_가락시장_data, on='일자')
무_통합=pd.merge(무_통합,공휴일,on='일자')
무_통합=pd.merge(무_통합,서울날씨,on='일자')
무_통합=pd.merge(무_통합,무_도매,on='일자')
무_통합.to_excel('무_시장.xlsx')
무_통합.info()

In [ ]:
최저시급=pd.read_excel('/content/최저시급.xlsx')
# 가격데이터 일자 컬럼 -> 날짜데이터로 변환
최저시급['일자'] = 최저시급.일자.astype(str)
최저시급['일자'] = pd.to_datetime(최저시급['일자'])
무_통합=pd.merge(무_통합, 최저시급,  on='일자')

In [ ]:
무_통합=무_통합[['최저시급','가격','반입량','경락가평균가격','유가 전국평균가격','서울평균도매가격','유무','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
무_통합.data=무_통합[['반입량','경락가평균가격','유가 전국평균가격','서울평균도매가격','최저시급']]
#,'일강수량(mm)','최저기온(°C)','최고기온(°C)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)',

In [ ]:
무_통합.target=무_통합[['가격']]
x,y=무_통합.data,무_통합.target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)
xgbr.fit(xtrain, ytrain)
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

# 소비자가격 예측 머신러닝(1)

#### 오이(시장) 소비자가 전처리 및 예측

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
오이_도매가격=pd.read_excel('/content/서울 평균 도매가격(오이).xlsx')
오이_시장마트가격=pd.read_excel('/content/서울전역시장마트데이터(구분포함,23월포함).xlsx')
오이_경락가=pd.read_excel('/content/오이_경락가.xlsx')
오이_반입량=pd.read_excel('/content/오이_반입량.xlsx')
유가가격=pd.read_excel('/content/주유소_평균판매가격.xlsx')
서울날씨=pd.read_excel('/content/seoul_weather.xlsx')

In [ ]:
오이_도매가격=오이_도매가격[['date','wholesale_price(1pc)']]
오이_도매가격.rename(columns={'date':'일자'},inplace=True)
오이_시장가격=오이_시장마트가격[(오이_시장마트가격['품목']=='오이')&(오이_시장마트가격['구분']=='시장')]
오이_시장가격=오이_시장가격[['조사일','가격']]
오이_시장가격=오이_시장가격.groupby('조사일').mean()
오이_시장가격.reset_index('조사일',inplace=True)
오이_시장가격.rename(columns={'조사일':'일자'},inplace=True)
오이_시장가격.head()

In [ ]:
오이_경락가.head()
오이_경락가['일자'] = 오이_경락가.일자.astype(str)
오이_경락가['일자'] = pd.to_datetime(오이_경락가['일자'])
오이_경락가.rename(columns={'오이 1개 평균가격':'오이1개 경락가'},inplace=True)
오이_경락가=오이_경락가[['일자','평균가격(1)']]

In [ ]:
오이_반입량.head()
오이_반입량.rename(columns={'거래일자':'일자'},inplace=True)
오이_반입량['일자'] = pd.to_datetime(오이_반입량['일자'])
오이_반입량=오이_반입량[['일자','반입량']]
#오이_반입량.head()

In [ ]:
#유가가격.head()

In [ ]:
서울날씨.rename(columns={'일시':'일자'},inplace=True)
오이_시장=pd.merge(오이_시장가격,오이_도매가격,on='일자')
오이_시장=pd.merge(오이_시장,오이_경락가,on='일자')
오이_시장=pd.merge(오이_반입량,오이_시장,on='일자')
오이_시장=pd.merge(유가가격,오이_시장,on='일자')
오이_시장=pd.merge(오이_시장,서울날씨,on='일자')
오이_시장.to_excel('오이_시장.xlsx')

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
오이_시장=pd.read_excel('/content/오이_시장.xlsx')

In [ ]:
x_data=오이_시장[['유가 전국평균가격','반입량','wholesale_price(1pc)','평균가격(1)','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]
y_target=오이_시장[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)
# 변수 다있었을 때
# x_data=오이_시장[['유가 전국평균가격','반입량','wholesale_price(1pc)','평균가격(1)','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]
# 0.83
# 날씨 변수 제외하면?
#x_data=오이_시장[['유가 전국평균가격','반입량','wholesale_price(1pc)','오이1개 경락가']]
# 0.77

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
xgbr.fit(xtrain, ytrain)

In [ ]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain)
print("Mean cross-validation score: %.2f" % scores.mean())
#kfold = KFold(n_splits=8, shuffle=True)
#kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
#print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

In [ ]:
rf=RandomForestRegressor(random_state=0,n_estimators=1000)
neg_mse_scores=cross_val_score(rf,x_data,y_target,scoring='neg_mean_squared_error',cv=5)
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse=np.mean(rmse_scores)

print('5교차의 개별 Negative MSE scores : ',np.round(neg_mse_scores,2))
print('5교차 검증의 개별 RMSE score : ',np.round(rmse_scores,2))
print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))

In [ ]:
def get_model_cv_prediction(model, x_data,y_target):
    neg_mse_scores=cross_val_score(model,x_data,y_target,scoring='neg_mean_squared_error',cv=5)
    rmse_scores=np.sqrt(-1*neg_mse_scores)
    avg_rmse=np.mean(rmse_scores)
    print('####',model.__class__.__name__,'####')
    print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))
    score = xgbr.score(xtrain, ytrain)  
    print("Training score: ", score)
    scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
    print("Mean cross-validation score: %.2f" % scores.mean())
    kfold = KFold(n_splits=10, shuffle=True)
    kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
    print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_reg=DecisionTreeRegressor(random_state=0,max_depth=4)
rf_reg=RandomForestRegressor(random_state=0,n_estimators=1000)
gb_reg=GradientBoostingRegressor(random_state=0,n_estimators=1000)
xgb_reg=XGBRegressor(n_estimators=1000)
lgb_reg=LGBMRegressor(n_estimators=1000)

# 트리기반의 회귀 모델을 반복하면서 평가 수행
models=[dt_reg,rf_reg,gb_reg,xgb_reg,lgb_reg]
for model in models:
    get_model_cv_prediction(model, x_data, y_target)

In [ ]:
import seaborn as sns
%matplotlib inline

rf_reg=RandomForestRegressor(n_estimators=1000)

rf_reg.fit(x_data, y_target)

feature_series=pd.Series(data=rf_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
import seaborn as sns
%matplotlib inline

xgb_reg=XGBRegressor(n_estimators=1000)

xgb_reg.fit(x_data, y_target)

feature_series=pd.Series(data=xgb_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

#### 오이(마트) 소비자가 전처리 및 예측

In [ ]:
오이_마트가격=오이_시장마트가격[(오이_시장마트가격['품목']=='오이')&(오이_시장마트가격['구분']=='마트')]
오이_마트가격=오이_마트가격[['조사일','가격']]
오이_마트가격=오이_마트가격.groupby('조사일').mean()
오이_마트가격.reset_index('조사일',inplace=True)
오이_마트가격.rename(columns={'조사일':'일자'},inplace=True)
오이_마트=pd.merge(오이_마트가격,오이_도매가격,on='일자')
오이_마트=pd.merge(오이_마트,오이_경락가,on='일자')
오이_마트=pd.merge(오이_반입량,오이_마트,on='일자')
오이_마트=pd.merge(유가가격,오이_마트,on='일자')
오이_마트=pd.merge(오이_마트,서울날씨,on='일자')
오이_마트.to_excel('오이_마트.xlsx')

In [ ]:
x_data=오이_마트[['유가 전국평균가격','반입량','wholesale_price(1pc)','평균가격(1)','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)']]
y_target=오이_마트[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)
xgbr = xgb.XGBRegressor(verbosity=0) 

In [ ]:
xgbr.fit(xtrain, ytrain)

In [ ]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

#### 상추(시장) 소비자가 전처리 및 예측

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
상추_시장=pd.read_excel('/content/상추_시장_통합본.xlsx')

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
상추_시장.dropna(inplace=True)

In [ ]:
x_data=상추_시장[['반입량','100g가격','최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)','여부','유가 전국평균가격',	'도매가격']]
y_target=상추_시장[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
xgbr.fit(xtrain, ytrain)
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

# 소비자가격 예측 머신러닝(2)

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
무_시장=pd.read_excel('/content/무_시장.xlsx')

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [ ]:
무_시장.head()

In [ ]:
x_data=무_시장[['경락가평균가격','반입량','유가 전국평균가격','유무','최저기온(°C)','최고기온(°C)','일강수량(mm)','도매가격']]
y_target=무_시장[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import math

In [ ]:
sns.distplot(y_target, rug=True)
y_target=np.log1p(y_target)
y_target.hist()
무_시장['가격']=np.log1p(무_시장['가격'])
sns.distplot(무_시장['가격'],fit=norm)
stats.probplot(무_시장['가격'], plot=plt)

##### XGBoost

In [ ]:
무_시장=무_시장.drop(['Unnamed: 0','일자', '일교차','유무' ,'최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)'],axis=1)

In [ ]:
x_data=무_시장[['반입량','경락가평균가격','유가 전국평균가격','서울평균도매가격']]
y_target=무_시장[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
sns.distplot(무_시장['가격'],fit=norm)
stats.probplot(무_시장['가격'],plot=plt)
무_시장['가격']=np.log1p(무_시장['가격'])
sns.distplot(무_시장['가격'],fit=norm)
stats.probplot(무_시장['가격'], plot=plt)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg', random_state=100,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
xgbr.fit(xtrain, ytrain)

In [ ]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

#####RandomForestRegressor

In [ ]:
rf=RandomForestRegressor(random_state=0,n_estimators=1000)
neg_mse_scores=cross_val_score(rf,x_data,y_target,scoring='neg_mean_squared_error')
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse=np.mean(rmse_scores)

print('5교차의 개별 Negative MSE scores : ',np.round(neg_mse_scores,2))
print('5교차 검증의 개별 RMSE score : ',np.round(rmse_scores,2))
print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))

In [ ]:
def get_model_cv_prediction(model, x_data,y_target):
    neg_mse_scores=cross_val_score(model,x_data,y_target,scoring='neg_mean_squared_error',cv=5)
    rmse_scores=np.sqrt(-1*neg_mse_scores)
    avg_rmse=np.mean(rmse_scores)
    print('####',model.__class__.__name__,'####')
    print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))
    score = xgbr.score(xtrain, ytrain)  
    print("Training score: ", score)
    scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
    print("Mean cross-validation score: %.2f" % scores.mean())

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_reg=DecisionTreeRegressor(random_state=0,max_depth=4)
rf_reg=RandomForestRegressor(random_state=0,n_estimators=1000)
gb_reg=GradientBoostingRegressor(random_state=0,n_estimators=1000)
xgb_reg=XGBRegressor(n_estimators=1000)
lgb_reg=LGBMRegressor(n_estimators=1000)

# 트리기반의 회귀 모델을 반복하면서 평가 수행
models=[dt_reg,rf_reg,gb_reg,xgb_reg,lgb_reg]
for model in models:
    get_model_cv_prediction(model, x_data, y_target)

In [ ]:
import seaborn as sns
%matplotlib inline

rf_reg=RandomForestRegressor(n_estimators=1000)

rf_reg.fit(x_data, y_target)

feature_series=pd.Series(data=rf_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
import seaborn as sns
%matplotlib inline

xgb_reg=XGBRegressor(n_estimators=1000)

xgb_reg.fit(x_data, y_target)

feature_series=pd.Series(data=xgb_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

### BaggingRegressor

In [ ]:
무_시장.head()
data=무_시장.drop(['Unnamed: 0','일자', '일교차','유무' ,'최저기온(°C)','최고기온(°C)','일강수량(mm)','최대 풍속(m/s)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','평균 5cm 지중온도(°C)'],axis=1)

In [ ]:
data['가격']=np.log1p(data['가격'])
sns.distplot(data['가격'],fit=norm)
stats.probplot(data['가격'], plot=plt)

In [ ]:
feature_columns=list(data.columns.difference(['가격']))
X=data[feature_columns]
y=data['가격']
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=0.15,random_state=42)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
# 학습데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증

In [ ]:
from sklearn.linear_model import LinearRegression
from math import sqrt, log

In [ ]:
regression_model=LinearRegression()
linear_model1=regression_model.fit(train_x,train_y)
predict1=linear_model1.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict1,test_y))))

In [ ]:
from sklearn.ensemble import BaggingRegressor
bagging_model=BaggingRegressor(base_estimator=regression_model,
                               n_estimators=10,
                               verbose=1)
linear_model2=bagging_model.fit(train_x,train_y)
predict2=linear_model2.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict2,test_y))))

In [ ]:
bagging_mode2=BaggingRegressor(base_estimator=regression_model,
                               n_estimators=30,
                               verbose=1)
linear_model3=bagging_mode2.fit(train_x,train_y)
predict3=linear_model3.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict3,test_y))))

In [ ]:
# DecisionTreeRegressor
# BaggingRegressor에서  base_estimator를 decision_tree_model로 변경 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model=DecisionTreeRegressor()
tree_model1=decision_tree_model.fit(train_x,train_y)
predict1=tree_model1.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict1,test_y))))

In [ ]:
bagging_decision_model1=BaggingRegressor(base_estimator=decision_tree_model,
                               n_estimators=10,
                               verbose=1)
tree_model2=bagging_decision_model1.fit(train_x,train_y)
predict2=tree_model2.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict2,test_y))))

In [ ]:
bagging_decision_model1=BaggingRegressor(base_estimator=decision_tree_model,
                               n_estimators=30,
                               verbose=1)
tree_model2=bagging_decision_model1.fit(train_x,train_y)
predict2=tree_model2.predict(test_x)
print('RMSE:{}'.format(sqrt(mean_squared_error(predict2,test_y))))

### 예측 모델링

* DecisionTreeRegressor
* RandomForestRegressor
* GradientBoostingRegressor
* XGBRegressor
* LGBMRegressor







In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_reg=DecisionTreeRegressor(random_state=0,max_depth=4)
rf_reg=RandomForestRegressor(random_state=0,n_estimators=1000)
gb_reg=GradientBoostingRegressor(random_state=0,n_estimators=1000)
xgb_reg=XGBRegressor(n_estimators=1000)
lgb_reg=LGBMRegressor(n_estimators=1000)

# 트리기반의 회귀 모델을 반복하면서 평가 수행
models=[dt_reg,rf_reg,gb_reg,xgb_reg,lgb_reg]
for model in models:
    get_model_cv_prediction(model, x_data, y_target)

In [ ]:
import seaborn as sns
%matplotlib inline

rf_reg=RandomForestRegressor(n_estimators=1000)

rf_reg.fit(x_data, y_target)

feature_series=pd.Series(data=rf_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
import seaborn as sns
%matplotlib inline

xgb_reg=XGBRegressor(n_estimators=1000)

xgb_reg.fit(x_data, y_target)

feature_series=pd.Series(data=xgb_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

# 최종모델 선정

## hyper_Parameter(무_시장)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# 코랩 한글깨짐 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
dataset = pd.read_excel('/content/무_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.head()
# dataset.corr()

In [ ]:
dataset['가격']= np.log1p(dataset['가격'])
dataset['경락가평균가격']= np.log1p(dataset['경락가평균가격'])
dataset['반입량']= np.log1p(dataset['반입량'])
dataset['유가 전국평균가격']= np.log1p(dataset['유가 전국평균가격'])
dataset['도매가격']= np.log1p(dataset['도매가격'])

In [ ]:
dataset1=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset1.head()

In [ ]:
dataset2=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset2[:]=scalar.fit_transform(dataset2[:])

In [ ]:
dataset2.head()

In [ ]:
dataset=pd.concat([dataset1,dataset2],axis=1)

In [ ]:
dataset.head()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

##### ParameterTuning

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:
gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.7, 
                       gamma = 0.1,
                       learning_rate = 0.03,
                       max_depth = 3,
                       min_child_weight = 5,
                       n_estimators = 200,
                       objective = 'reg:squarederror',
                       subsample = 0.7)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 7,
                               max_features = 3,
                               min_samples_leaf = 3,
                               min_samples_split = 8,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.7,
                        gamma = 0.1,
                        learning_rate = 0.1,
                        max_depth = 7,
                        min_child_samples = 20,
                        n_estimators = 100,
                        num_leaves =8,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.1,
                                   max_depth=7,
                                   n_estimators=100,
                                   subsample=0.7
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
#ypred = rf_reg.predict(X_test)
#mse = mean_squared_error(y_test, ypred)

In [ ]:
#x_ax = range(len(y_test))
#plt.plot(x_ax, y_test, label="Original Price")
#plt.plot(x_ax, ypred, label="Predicted Price")
#plt.title("Price Prediction")
#plt.legend()
#plt.show()

In [ ]:
from sklearn.ensemble import VotingRegressor

er = VotingRegressor([('xgb_reg', xgb_reg), ('gb_reg', gb_reg),('lgb_reg',lgb_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

##### 예측

In [ ]:
##########예측

In [ ]:
무_더미= pd.read_excel('/content/무_더미(예측).xlsx')

In [ ]:
#무_더미['가격']= np.log1p(무_더미['가격'])
무_더미['경락가평균가격']= np.log1p(무_더미['경락가평균가격'])
무_더미['반입량']= np.log1p(무_더미['반입량'])
무_더미['유가 전국평균가격']= np.log1p(무_더미['유가 전국평균가격'])
무_더미['도매가격']= np.log1p(무_더미['도매가격'])

In [ ]:
무_더미1=무_더미[['경락가평균가격','반입량','유가 전국평균가격','도매가격']]
무_더미1.head()

In [ ]:
무_더미2=무_더미[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
무_더미2[:]=scalar.fit_transform(무_더미2[:])

In [ ]:
무_더미2.head()

In [ ]:
무_더미=pd.concat([무_더미1,무_더미2],axis=1)

In [ ]:
ypred = er.predict(무_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
무_더미['예측가격'] = ypred

In [ ]:
무_더미['예측가격']=np.expm1(무_더미['예측가격'])

In [ ]:
무_더미.to_excel('무_더미(예측완료).xlsx')

## hyper_Parameter(배추_시장)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
dataset = pd.read_excel('/content/배추_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.head()
# dataset.corr()

In [ ]:
dataset.info()

In [ ]:
###예측##

In [ ]:
dataset['가격']= np.log1p(dataset['가격'])
dataset['경락가평균가격']= np.log1p(dataset['경락가평균가격'])
dataset['반입량']= np.log1p(dataset['반입량'])
dataset['유가 전국평균가격']= np.log1p(dataset['유가 전국평균가격'])
dataset['도매가격']= np.log1p(dataset['도매가격'])

In [ ]:
dataset1=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset1.head()

In [ ]:
dataset2=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset2[:]=scalar.fit_transform(dataset2[:])

In [ ]:
dataset2.head()

In [ ]:
dataset=pd.concat([dataset1,dataset2],axis=1)

In [ ]:
dataset.head()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

#### Parameter Tuning

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.7, 
                       gamma = 0.2,
                       learning_rate = 0.1,
                       max_depth = 5,
                       min_child_weight = 1,
                       n_estimators = 500,
                       objective = 'reg:squarederror',
                       subsample = 0.7)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 10,
                               max_features = 3,
                               min_samples_leaf = 3,
                               min_samples_split = 8,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.7,
                        gamma = 0.1,
                        learning_rate = 0.03,
                        max_depth = 7,
                        min_child_samples = 10,
                        n_estimators = 200,
                        num_leaves =8,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.1,
                                   max_depth=3,
                                   n_estimators=200,
                                   subsample=0.5
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingRegressor

er = VotingRegressor([('xgb_reg', xgb_reg), ('gb_reg', gb_reg),('lgb_reg',lgb_reg),('rf_reg',rf_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

#### 예측

In [ ]:
배추_더미= pd.read_excel('/content/배추_더미(예측).xlsx')

In [ ]:
#무_더미['가격']= np.log1p(무_더미['가격'])
배추_더미['경락가평균가격']= np.log1p(배추_더미['경락가평균가격'])
배추_더미['반입량']= np.log1p(배추_더미['반입량'])
배추_더미['유가 전국평균가격']= np.log1p(배추_더미['유가 전국평균가격'])
배추_더미['도매가격']= np.log1p(배추_더미['도매가격'])

In [ ]:
배추_더미1=배추_더미[['경락가평균가격','반입량','유가 전국평균가격','도매가격']]
배추_더미1.head()

In [ ]:
배추_더미2=배추_더미[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
배추_더미2[:]=scalar.fit_transform(배추_더미2[:])

In [ ]:
배추_더미2.head()

In [ ]:
배추_더미=pd.concat([배추_더미1,배추_더미2],axis=1)

In [ ]:
ypred = gb_reg.predict(배추_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
배추_더미['예측가격'] = ypred

In [ ]:
배추_더미.head()

In [ ]:
배추_더미['예측가격']=np.expm1(배추_더미['예측가격'])

In [ ]:
배추_더미.to_excel('배추_더미(예측완료).xlsx')

In [ ]:
abc = []
for alg in er.named_estimators:
    clf = er.named_estimators[alg]
    a = clf.__class__.__name__
    b = [pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])]
    abc.append({a:b})

abc

## hyper_Parameter(상추_시장)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

상추

In [ ]:
dataset = pd.read_excel('/content/상추_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.rename(columns={'경락가평균가겨':'경락가평균가격'},inplace=True)
dataset.head()
# dataset.corr()

In [ ]:
dataset.info()

In [ ]:
dataset2=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset2.head() #위 변수들만 dataset2로 지정 log 변환해줌

dataset2['가격']= np.log1p(dataset2['가격'])
dataset2['경락가평균가격']= np.log1p(dataset2['경락가평균가격'])
dataset2['반입량']= np.log1p(dataset2['반입량'])
dataset2['유가 전국평균가격']= np.log1p(dataset2['유가 전국평균가격'])
dataset2['도매가격']= np.log1p(dataset2['도매가격'])

dataset1=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset1[:]=scalar.fit_transform(dataset1[:]) #전부다 MinMax로 

dataset=pd.concat([dataset2,dataset1],axis=1)

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

#### Parameter Tuning

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.7, 
                       gamma = 0.1,
                       learning_rate = 0.03,
                       max_depth = 5,
                       min_child_weight = 1,
                       n_estimators = 200,
                       objective = 'reg:squarederror',
                       subsample = 0.7)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 7,
                               max_features = 3,
                               min_samples_leaf = 3,
                               min_samples_split = 8,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.7,
                        gamma = 0.1,
                        learning_rate = 0.03,
                        max_depth = 5,
                        min_child_samples = 10,
                        n_estimators = 200,
                        num_leaves =10,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.01,
                                   max_depth=5,
                                   n_estimators=200,
                                   subsample=0.7
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingRegressor

er = VotingRegressor([('xgb_reg', xgb_reg), ('gb_reg', gb_reg),('rf_reg',rf_reg),('lgb_reg',lgb_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

#### 예측

In [ ]:
상추_더미= pd.read_excel('/content/상추_더미(예측).xlsx')

In [ ]:
#무_더미['가격']= np.log1p(무_더미['가격'])
상추_더미['경락가평균가격']= np.log1p(상추_더미['경락가평균가격'])
상추_더미['반입량']= np.log1p(상추_더미['반입량'])
상추_더미['유가 전국평균가격']= np.log1p(상추_더미['유가 전국평균가격'])
상추_더미['도매가격']= np.log1p(상추_더미['도매가격'])

In [ ]:
상추_더미1=상추_더미[['경락가평균가격','반입량','유가 전국평균가격','도매가격']]
상추_더미1.head()

In [ ]:
상추_더미2=상추_더미[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
상추_더미2[:]=scalar.fit_transform(상추_더미2[:])

In [ ]:
상추_더미2.head()

In [ ]:
상추_더미=pd.concat([상추_더미1,상추_더미2],axis=1)

In [ ]:
ypred = er.predict(상추_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
상추_더미['예측가격']=ypred

In [ ]:
상추_더미['예측가격']=np.expm1(상추_더미['예측가격'])

In [ ]:
abc = []
for alg in er.named_estimators:
    clf = er.named_estimators[alg]
    a = clf.__class__.__name__
    b = [pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])]
    abc.append({a:b})

abc

##hyper_Parameter(오이_시장)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

오이

In [ ]:
dataset = pd.read_excel('/content/오이_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.head()
# dataset.corr()

In [ ]:
dataset2=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset2.head() #위 변수들만 dataset2로 지정 log 변환해줌

dataset2['가격']= np.log1p(dataset2['가격'])
dataset2['경락가평균가격']= np.log1p(dataset2['경락가평균가격'])
dataset2['반입량']= np.log1p(dataset2['반입량'])
dataset2['유가 전국평균가격']= np.log1p(dataset2['유가 전국평균가격'])
dataset2['도매가격']= np.log1p(dataset2['도매가격'])

dataset1=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset1[:]=scalar.fit_transform(dataset1[:]) #전부다 MinMax로 

dataset=pd.concat([dataset1,dataset2],axis=1)

In [ ]:
dataset.info()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

#### Parameter Tuning

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.5, 
                       gamma = 0.1,
                       learning_rate = 0.01,
                       max_depth = 7,
                       min_child_weight = 3,
                       n_estimators = 500,
                       objective = 'reg:squarederror',
                       subsample = 0.5)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 10,
                               max_features = 3,
                               min_samples_leaf = 3,
                               min_samples_split = 10,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.7,
                        gamma = 0.1,
                        learning_rate = 0.03,
                        max_depth = 5,
                        min_child_samples = 10,
                        n_estimators =100,
                        num_leaves =10,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.05,
                                   max_depth=3,
                                   n_estimators=200,
                                   subsample=0.5
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingRegressor

er = VotingRegressor([('xgb_reg', xgb_reg), ('gb_reg', gb_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

#### 예측

In [ ]:
오이_더미= pd.read_excel('/content/오이_더미(예측).xlsx')

In [ ]:
오이_더미.info()

In [ ]:
#무_더미['가격']= np.log1p(무_더미['가격'])
오이_더미['경락가평균가격']= np.log1p(오이_더미['경락가평균가격'])
오이_더미['반입량']= np.log1p(오이_더미['반입량'])
오이_더미['유가 전국평균가격']= np.log1p(오이_더미['유가 전국평균가격'])
오이_더미['도매가격']= np.log1p(오이_더미['도매가격 '])

In [ ]:
오이_더미1=오이_더미[['경락가평균가격','반입량','유가 전국평균가격','도매가격']]
오이_더미1.head()

In [ ]:
오이_더미2=오이_더미[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
오이_더미2[:]=scalar.fit_transform(오이_더미2[:])

In [ ]:
오이_더미2.head()

In [ ]:
오이_더미=pd.concat([오이_더미2,오이_더미1],axis=1)

In [ ]:
오이_더미

In [ ]:
ypred = gb_reg.predict(오이_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
오이_더미['예측가격'] = ypred

In [ ]:
오이_더미

In [ ]:
오이_더미['예측가격']=np.expm1(오이_더미['예측가격'])

In [ ]:
오이_더미

In [ ]:
오이_더미.to_excel('오이_더미(예측완료).xlsx')

In [ ]:
pd.DataFrame(sorted(zip(gb_reg.feature_importances_,X_train.columns)), columns=['Value','Feature'])

##hyper_Parameter(양파_시장)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

양파

In [ ]:
dataset = pd.read_excel('/content/양파_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.head()
# dataset.corr()

In [ ]:
dataset2=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset2.head() #위 변수들만 dataset2로 지정 log 변환해줌

dataset2['가격']= np.log1p(dataset2['가격'])
dataset2['경락가평균가격']= np.log1p(dataset2['경락가평균가격'])
dataset2['반입량']= np.log1p(dataset2['반입량'])
dataset2['유가 전국평균가격']= np.log1p(dataset2['유가 전국평균가격'])
dataset2['도매가격']= np.log1p(dataset2['도매가격'])

dataset1=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset1[:]=scalar.fit_transform(dataset1[:]) #전부다 MinMax로 

dataset=pd.concat([dataset1,dataset2],axis=1)
dataset.head()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
#sns.distplot(y_target, rug=True) 

In [ ]:
#dataset['가격']=np.log1p(dataset['가격'])
#sns.distplot(dataset['가격'],fit=norm)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

#### Parameter Tuning

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.5, 
                       gamma = 0.1,
                       learning_rate = 0.01,
                       max_depth = 3,
                       min_child_weight = 5,
                       n_estimators = 500,
                       objective = 'reg:squarederror',
                       subsample = 0.5)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 7,
                               max_features = 2,
                               min_samples_leaf = 5,
                               min_samples_split = 12,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.5,
                        gamma = 0.1,
                        learning_rate = 0.01,
                        max_depth = 5,
                        min_child_samples = 10,
                        n_estimators = 200,
                        num_leaves =6,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.01,
                                   max_depth=3,
                                   n_estimators=200,
                                   subsample=0.5
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

#rf_reg = RandomForestRegressor(random_state=0, 
#                              bootstrap = [True],
#                              max_depth = 7,
#                              max_features = 2,
#                               min_samples_leaf = 5,
#                               min_samples_split = 12,
#                               n_estimators = 100
#                               )

#gb_reg = GradientBoostingRegressor(random_state=0, 
#                                   learning_rate=0.01,
#                                   max_depth=3,
#                                   n_estimators=200,
#                                   subsample=0.5
#                                   )
er = VotingRegressor([('xgb_reg', xgb_reg), ('rf_reg', rf_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
#y_pred = er.fit(X_data, y_target).predict(X_data) 

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

#### 예측

In [ ]:
양파_더미= pd.read_excel('/content/양파_더미(예측).xlsx')

In [ ]:
#무_더미['가격']= np.log1p(무_더미['가격'])
양파_더미['경락가평균가격']= np.log1p(양파_더미['경락가평균가격'])
양파_더미['반입량']= np.log1p(양파_더미['반입량'])
양파_더미['유가 전국평균가격']= np.log1p(양파_더미['유가 전국평균가격'])
양파_더미['도매가격']= np.log1p(양파_더미['도매가격'])

In [ ]:
양파_더미1=양파_더미[['경락가평균가격','반입량','유가 전국평균가격','도매가격']]
양파_더미1.head()

In [ ]:
양파_더미2=양파_더미[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
양파_더미2[:]=scalar.fit_transform(양파_더미2[:])

In [ ]:
양파_더미2.head()

In [ ]:
양파_더미=pd.concat([양파_더미2,양파_더미1],axis=1)

In [ ]:
양파_더미

In [ ]:
ypred = er.predict(양파_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
양파_더미['예측가격'] = ypred

In [ ]:
양파_더미

In [ ]:
양파_더미['예측가격']=np.expm1(양파_더미['예측가격'])

In [ ]:
양파_더미

In [ ]:
양파_더미.to_excel('양파_더미(예측완료).xlsx')

In [ ]:
abc = []
for alg in er.named_estimators:
    clf = er.named_estimators[alg]
    a = clf.__class__.__name__
    b = [pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])]
    abc.append({a:b})

abc